In [0]:

!pip install pandas-profiling
!pip install pysqlite3 
!pip install pandas
!pip install typing-extensions==4.12.2

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import sqlite3  
import pandas as pd  
  
conn = sqlite3.connect('customer_data.sqlite3')  


In [0]:
def load_api_oldcustomer(conn):  
    query = '''  
    SELECT loan_status,  
           loan_amnt,  
           term,  
           CAST(int_rate AS REAL) as int_rate,  
           CAST(installment AS REAL) as installment,  
           sub_grade,  
           CASE  
               WHEN emp_length = '10+ years' THEN 10  
               WHEN emp_length = '< 1 year' THEN 0  
               ELSE CAST(SUBSTR(emp_length, 1, 2) AS INTEGER)  
           END as emp_length,  
           home_ownership,  
           CASE  
               WHEN home_ownership = 'MORTGAGE' THEN 1  
               ELSE 0  
           END as is_mortgage,  
           CASE  
               WHEN home_ownership = 'RENT' THEN 1  
               ELSE 0  
           END as is_rent,  
           CASE  
               WHEN home_ownership = 'OWN' THEN 1  
               ELSE 0  
           END as is_own,  
           CASE  
               WHEN home_ownership = 'ANY' THEN 1  
               ELSE 0  
           END as is_any,  
           CASE  
               WHEN home_ownership = 'OTHER' THEN 1  
               ELSE 0  
           END as is_other,  
           CAST(annual_inc AS INTEGER) as annual_inc,  
           verification_status,  
           CASE  
               WHEN verification_status = 'Verified' THEN 1  
               ELSE 0  
           END as is_verified,  
           CASE  
               WHEN verification_status = 'Not Verified' THEN 1  
               ELSE 0  
           END as is_not_verified,  
           CASE  
               WHEN verification_status = 'Source Verified' THEN 1  
               ELSE 0  
           END as is_source_verified,  
           issue_d,  
           purpose,  
           addr_state,  
           CAST(dti AS REAL) as dti,  
           fico_range_low,  
           fico_range_high,  
           open_acc,  
           pub_rec,  
           revol_bal,  
           CAST(revol_util AS REAL) as revol_util,  
           mort_acc,  
           pub_rec_bankruptcies,  
           CAST(age AS INTEGER) as age,  
           pay_status  
    FROM api_oldcustomer  
    '''  
    df = pd.read_sql(query, conn)  
  
    # Fill missing values with median or mode  
    df['dti'].fillna(df['dti'].median(), inplace=True)  
    df['revol_util'].fillna(df['revol_util'].median(), inplace=True)  
    df['mort_acc'].fillna(df['mort_acc'].mode()[0], inplace=True)  
    df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].mode()[0], inplace=True)  
  
    return df  


In [0]:
def load_api_newcustomer(conn):  
    query = '''  
    SELECT loan_status,  
           CAST(loan_amnt AS REAL) as loan_amnt,  
           term,  
           CAST(int_rate AS REAL) as int_rate,  
           CAST(installment AS REAL) as installment,  
           sub_grade_id,  
           employment_length,  
           home_ownership_id,  
           CAST(annual_inc AS INTEGER) as annual_inc,  
           verification_status_id,  
           issued,  
           purpose_id,  
           addr_state_id,  
           CAST(dti AS REAL) as dti,  
           CAST(fico_range_low AS INTEGER) as fico_range_low,  
           CAST(fico_range_high AS INTEGER) as fico_range_high,  
           CAST(open_acc AS INTEGER) as open_acc,  
           CAST(pub_rec AS INTEGER) as pub_rec,  
           CAST(revol_bal AS INTEGER) as revol_bal,  
           CAST(revol_util AS REAL) as revol_util,  
           CAST(mort_acc AS INTEGER) as mort_acc,  
           CAST(pub_rec_bankruptcies AS INTEGER) as pub_rec_bankruptcies,  
           CAST(age AS INTEGER) as age,  
           payment_status  
    FROM api_newcustomer  
    '''  
    df = pd.read_sql(query, conn)  
  
    # Fill missing values with median or mode  
    df['dti'].fillna(df['dti'].median(), inplace=True)  
    df['revol_util'].fillna(df['revol_util'].median(), inplace=True)  
    df['mort_acc'].fillna(df['mort_acc'].mode()[0], inplace=True)  
    df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].mode()[0], inplace=True)  
  
    return df  


def load_api_homeownership(conn):  
    query = '''  
    SELECT id, name  
    FROM api_homeownership  
    '''  
    df = pd.read_sql(query, conn)  
    return df  
  
  
def load_api_purpose(conn):  
    query = '''  
    SELECT id, name  
    FROM api_purpose  
    '''  
    df = pd.read_sql(query, conn)  
    return df  
  
  
def load_api_state(conn):  
    query = '''  
    SELECT id, name  
    FROM api_state  
    '''  
    df = pd.read_sql(query, conn)  
    return df  
  
  
def load_api_subgrade(conn):  
    query = '''  
    SELECT id, name  
    FROM api_subgrade  
    '''  
    df = pd.read_sql(query, conn)  
    return df  

def load_api_verificationstatus(conn):  
    query = '''  
    SELECT id, name  
    FROM api_verificationstatus  
    '''  
    df = pd.read_sql(query, conn)  
    return df  



In [0]:
api_homeownership = pd.read_sql('SELECT * FROM api_homeownership', conn)  
api_purpose = pd.read_sql('SELECT * FROM api_purpose', conn)  
api_state = pd.read_sql('SELECT * FROM api_state', conn)  
api_subgrade = pd.read_sql('SELECT * FROM api_subgrade', conn)  
  
df = load_api_oldcustomer(conn)  
df = df.merge(api_homeownership, how='left', left_on='home_ownership', right_on='name')  
df = df.merge(api_purpose, how='left', left_on='purpose', right_on='name')  
df = df.merge(api_state, how='left', left_on='addr_state', right_on='name')  
df = df.merge(api_subgrade, how='left', left_on='sub_grade', right_on='name')  
  
df.drop(['home_ownership', 'purpose', 'addr_state', 'sub_grade'], axis=1, inplace=True)  


/root/.ipykernel/13731/command-1573740770627923-1457642408:10: FutureWarning: Passing 'suffixes' which cause duplicate columns {'name_x', 'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(api_subgrade, how='left', left_on='sub_grade', right_on='name')


In [0]:
df.to_csv('final_dataset.csv', index=False)  


In [0]:
import pandas as pd  
from pandas_profiling import ProfileReport  

df = pd.read_csv('final_dataset.csv')  


profile = ProfileReport(df, title='Profile Report', explorative=True)  

profile.to_file('profile_report.html')  


---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-1573740770627934>, line 2
      1 import pandas as pd  
----> 2 from pandas_profiling import ProfileReport  
      4 df = pd.read_csv('final_dataset.csv')  
      7 profile = ProfileReport(df, title='Profile Report', explorative=True)  

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d95e3ea6-5048-4c7f-9bc6-9fe6a41ab7f9/lib/python3.10/site-packages/pandas_profiling/__init__.py:8
      1 """Main module of pandas-profiling.
      2 
      3 .. include:: ../../README.md
      4 """
      6 from warnings import warn
----> 8 from pandas_profiling.compare_reports import compare
      9 from pandas_profiling.controller import pandas_decorator
     10 from pandas_profiling.profile_report import ProfileReport

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d95e3ea6-5048-4c7f-9bc6-9fe6a41ab7f9/lib/python3.10/site-packages/pandas_p